In [66]:
from transformers import BertModel, BertTokenizer, BertConfig, BertForSequenceClassification, AutoTokenizer, AutoModelForMaskedLM, BartTokenizer, BartForConditionalGeneration, AutoModelForSeq2SeqLM
import torch
import textwrap
import pandas as pd



tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')


In [74]:
with open('texts/FL_Brecko.txt', 'r') as file:
    text = file.read()

In [75]:
tokens = tokenizer.tokenize(text)
num_tokens = len(tokens)

print(f"Number of tokens in input text: {num_tokens}")

Number of tokens in input text: 37876


In [76]:
%%timeit

def chunk_text(text, max_tokens):
    words = text.split()
    chunks = []
    current_chunk = []

    for word in words:
        trial_text = ' '.join(current_chunk + [word])
        token_count = len(tokenizer.tokenize(trial_text))

        if token_count > max_tokens:
            chunks.append(' '.join(current_chunk))
            current_chunk = [word]
        else:
            current_chunk.append(word)

    if current_chunk:
        chunks.append(' '.join(current_chunk))
    
    return chunks


chunks = chunk_text(text, max_tokens=1024)

summaries = []
for chunk in chunks:
    inputs = tokenizer(chunk, return_tensors="pt", truncation=True, padding="max_length", max_length=1024)
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    summaries.append(summary)

final_summary = ' '.join(summaries)


2min 57s ± 191 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [65]:
csv_path = 'csv/SummaryText.csv'
df = pd.read_csv(csv_path)

new_data = pd.DataFrame({
    'Name of Article': ["SAMI2022"],
    'Summarized Text': [final_summary]
})

df = df.append(new_data, ignore_index=True)
df.to_csv(csv_path, index=False)


print("Summary saved to CSV successfully.")

Summary saved to CSV successfully.


/tmp/ipykernel_2786450/3200570203.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_data, ignore_index=True)
